##### This notebook has code of logistic regression, xgboost and mlp baselines for both valence and accuracy classification

In [1]:
cd C:\Users\Alfiya\Desktop\SLAB-main\Code

C:\Users\Alfiya\Desktop\SLAB-main\Code


In [3]:
import os
#Necessary packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os
import joblib 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import load_model

from supervised_models import logit, xgb_model, mlp
from utils import perf_metric
from VIME.vime_self import vime_self
from hexr_self import hexr_self

ModuleNotFoundError: No module named 'supervised_models'

In [3]:
def one_hot_encode(df, class_label):
    #one-hot encode class column
    ohe = OneHotEncoder()
    #Choose either class1 or class2 to select either valence or arousal
    df_ohe = pd.DataFrame(ohe.fit_transform(df[[class_label]]).toarray())
    df = df.join(df_ohe)
    
    df.drop('class1', axis=1, inplace=True)
    df.drop('class2', axis=1, inplace=True)
    df.drop('Unnamed: 0', axis=1, inplace=True)
    df.drop('valence', axis=1, inplace=True)
    df.drop('arousal', axis=1, inplace=True)
    
    X = df.loc[:,:'emg_trap']
    y = df.iloc[:,8:]
    return X, y

In [4]:
def split_data(X, y, label_data_rate, test_size=0.2):
    """
    Train-test split and train split into unlablled and labelled data without normalization
    if Test is 15% of the data
    10% of 85% = 8.5% :- Labelled dataset
    90% of 85% = 76.5% :- Unlabelled dataset
    
    if Test is 20% of the data 
    10% of 80% = 8% :- Labelled dataset
    90% of 80% = 72% :- Unlabelled dataset
    """
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=33) 
    
    #converting to numpy arrays
    x_train = x_train.iloc[:, :].values
    y_train = y_train.iloc[:, :].values
    x_test = x_test.iloc[:,:].values
    y_test = y_test.iloc[:,:].values
    x_train.shape, x_test.shape, y_train.shape, y_test.shape
    
    # Divide labeled and unlabeled data
    idx = np.random.permutation(len(y_train))
    
    # Label data : Unlabeled data = label_data_rate:(1-label_data_rate)
    label_idx = idx[:int(len(idx)*label_data_rate)]
    unlab_idx = idx[int(len(idx)*label_data_rate):]
    
    # Unlabeled data
    x_unlab = x_train[unlab_idx, :]
    
    # Labeled data
    x_train = x_train[label_idx, :] 
    y_train = y_train[label_idx, :]
    return x_unlab, x_train, x_test, y_train, y_test

In [4]:
#Experimental parameters
label_data_rate = 0.1

#Metrics
metric1 = 'acc'
metric2 = 'auc'

#parameters to train the encoder
p_m = 0.3
alpha = 2.0

## Valence Accuracy using Logistic regression 

In [6]:
#load case dataset
df = pd.read_csv("..\Data\CASE_2class.csv")

In [7]:
X, Y = one_hot_encode(df, 'class1')

In [8]:
X.shape, Y.shape

((1470960, 8), (1470960, 2))

In [9]:
x_unlab, x_train, x_test, y_train, y_test = split_data(X, Y, label_data_rate, test_size=0.15)

In [10]:
x_unlab.shape, x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1125285, 8), (125031, 8), (220644, 8), (125031, 2), (220644, 2))

In [47]:
#logistic regression
logistic_valence_acc = []
logistic_valence_auc = []
y_test_hat = logit(x_train, y_train, x_test)
logistic_valence_acc.append(perf_metric(metric1, y_test, y_test_hat))
logistic_valence_auc.append(perf_metric(metric2, y_test, y_test_hat))

In [48]:
logistic_valence_acc

[0.6900708834140062]

In [85]:
logistic_valence_auc

[0.5450630299376711]

## Arousal Accuracy using Logistic regression 

In [86]:
#load case dataset
df = pd.read_csv("..\Data\CASE_2class.csv")

In [87]:
X, Y = one_hot_encode(df, 'class2')

In [88]:
X.shape, Y.shape

((1470960, 8), (1470960, 2))

In [89]:
x_unlab, x_train, x_test, y_train, y_test = split_data(X, Y, label_data_rate, test_size=0.15)

In [90]:
x_unlab.shape, x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1125285, 8), (125031, 8), (220644, 8), (125031, 2), (220644, 2))

In [94]:
#logistic regression
logistic_arousal_acc = []
logistic_arousal_auc = []
y_test_hat = logit(x_train, y_train, x_test)
logistic_arousal_acc.append(perf_metric(metric1, y_test, y_test_hat))
logistic_arousal_auc.append(perf_metric(metric2, y_test, y_test_hat))

In [95]:
logistic_arousal_acc

[0.6805804825873353]

In [93]:
logistic_arousal_auc

[0.5619202841925236]

## XGBoost Model

### Valence Accuracy using XGBoost

In [5]:
#load case dataset
df = pd.read_csv("..\Data\CASE_2class.csv")

In [6]:
X, Y = one_hot_encode(df, 'class1')

In [11]:
#For valence
xgboost_valence_acc = []
xgboost_valence_auc = []

for i in range (0, 5):
    x_unlab, x_train, x_test, y_train, y_test = split_data(X, Y, label_data_rate, test_size=0.15)
    
    y_test_hat = xgb_model(x_train, y_train, x_test)
    xgboost_valence_acc.append(perf_metric(metric1, y_test, y_test_hat))
    xgboost_valence_auc.append(perf_metric(metric2, y_test, y_test_hat))

In [12]:
xgboost_valence_acc

[0.9180897735719077,
 0.9237323471293124,
 0.9228576349232247,
 0.9213756095792317,
 0.919589927666286]

In [13]:
xgboost_valence_auc

[0.9701262251350858,
 0.9706931109094453,
 0.9715748127704218,
 0.9713079651254548,
 0.9705962091725275]

In [16]:
#Average Valence Accuracy using XGBoost model
print("Average Valence Accuracy on CASE dataset using XGBoost Model is: ", round(np.mean(xgboost_valence_acc),4))

print("Average Valence Accuracy  AUC SCore on CASE dataset using XGBoost Model is: ", round(np.mean(xgboost_valence_auc),4))

print("Standard Deviation of arousal Accuracies using SLAB: ", round(np.std(xgboost_valence_acc),4))

Average Valence Accuracy on CASE dataset using XGBoost Model is:  0.9211
Average Valence Accuracy  AUC SCore on CASE dataset using XGBoost Model is:  0.9709
Standard Deviation of arousal Accuracies using SLAB:  0.0021


### Arousal Accuracy using XGBoost

In [55]:
#load case dataset
df = pd.read_csv("..\Data\CASE_2class.csv")

In [57]:
X, Y = one_hot_encode(df, 'class2')

In [58]:
#For valence
xgboost_arousal_acc = []
xgboost_arousal_auc = []

for i in range (0, 5):
    x_unlab, x_train, x_test, y_train, y_test = split_data(X, Y, label_data_rate, test_size=0.15)
    
    y_test_hat = xgb_model(x_train, y_train, x_test)
    xgboost_arousal_acc.append(perf_metric(metric1, y_test, y_test_hat))
    xgboost_arousal_auc.append(perf_metric(metric2, y_test, y_test_hat))

In [59]:
xgboost_arousal_acc

[0.9134352169104983,
 0.9127010025198963,
 0.9120438353184315,
 0.9121933975091097,
 0.9186744257718316]

In [60]:
xgboost_arousal_auc

[0.9664071015620949,
 0.9652240022633946,
 0.9648969143594377,
 0.9640528755222806,
 0.969683191703373]

In [61]:
#Average Valence Accuracy using XGBoost model
print("Average Arousal Accuracy on CASE dataset using XGBoost Model is: ", np.mean(xgboost_arousal_acc))

Average Arousal Accuracy on CASE dataset using XGBoost Model is:  0.9138095756059534


## MLP Baseline

In [7]:
#load case dataset
df = pd.read_csv("..\Data\CASE_2class.csv")

In [12]:
mlp_parameters = dict()
mlp_parameters['hidden_dim'] = 100
mlp_parameters['epochs'] = 100
mlp_parameters['activation'] = 'relu'
mlp_parameters['batch_size'] = 128
mlp_parameters['num_layers'] = 2

### MLP Valence Accuracy for CASE dataset

In [9]:
X, y = one_hot_encode(df, 'class1')

In [10]:
x_unlab, x_train, x_test, y_train, y_test = split_data(X, y, label_data_rate, test_size=0.15)

In [11]:
mlp_valence_acc = []
mlp_valence_auc = []

for i in range (0, 5):
    y_test_hat = mlp(x_train, y_train, x_test, mlp_parameters)
    mlp_valence_acc.append(perf_metric(metric1, y_test, y_test_hat))
    mlp_valence_auc.append(perf_metric(metric2, y_test, y_test_hat))

In supervised_models.py!
6896/6896 [==============================] - 4s 564us/step
In supervised_models.py!
6896/6896 [==============================] - 4s 574us/step
In supervised_models.py!
6896/6896 [==============================] - 4s 561us/step
In supervised_models.py!
6896/6896 [==============================] - 4s 554us/step
In supervised_models.py!
6896/6896 [==============================] - 4s 563us/step


In [12]:
np.round(mlp_valence_acc,4)

array([0.8714, 0.8729, 0.8681, 0.8597, 0.8683])

In [13]:
np.round(mlp_valence_auc, 4)

array([0.9332, 0.9363, 0.9302, 0.9222, 0.9321])

In [18]:
print("Mean of valence accuracies for MLP is ",round(np.mean(mlp_valence_acc),4))

print("Mean of valence accuracies AUC Score for MLP is ",round(np.mean(mlp_valence_auc), 4))

# Compute standard deviation
std_dev_acc = np.std(mlp_valence_acc)
print("Standard Deviation of valence Accuracies using MLP baseline: ", std_dev_acc)

Mean of valence accuracies for MLP is  0.8681
Mean of valence accuracies AUC Score for MLP is  0.9308
Standard Deviation of valence Accuracies using MLP baseline:  0.004578423536709249


In [15]:
print("Mean of arousal accuracies for MLP is ",round(np.mean(mlp_valence_acc),4))

print("Mean of arousal accuracies AUC Score for MLP is ",round(np.mean(mlp_valence_auc), 4))

# Compute standard deviation
std_dev_acc = np.std(mlp_valence_acc)
print("Standard Deviation of arousal Accuracies using MLP baseline: ", std_dev_acc)

Mean of arousal accuracies for MLP is  0.87
Mean of arousal accuracies AUC Score for MLP is  0.9329
Standard Deviation of arousal Accuracies using MLP baseline:  0.0039567462732572


for 5 runs

In [14]:
print("Mean of arousal accuracies for MLP is ",round(np.mean(mlp_valence_acc),4))

print("Mean of arousal accuracies AUC Score for MLP is ",round(np.mean(mlp_valence_auc), 4))

# Compute standard deviation
std_dev_acc = np.std(mlp_valence_acc)
print("Standard Deviation of arousal Accuracies using MLP baseline: ", std_dev_acc)

Mean of arousal accuracies for MLP is  0.8695
Mean of arousal accuracies AUC Score for MLP is  0.9314
Standard Deviation of arousal Accuracies using MLP baseline:  0.004726429465268831


### MLP Arousal Accuracy for CASE dataset

In [15]:
X, Y = one_hot_encode(df, 'class2')

In [16]:
x_unlab, x_train, x_test, y_train, y_test = split_data(X, Y, label_data_rate, test_size=0.15)

In [17]:
mlp_arousal_acc = []
mlp_arousal_auc = []

for i in range (0, 5):
    y_test_hat = mlp(x_train, y_train, x_test, mlp_parameters)
    mlp_arousal_acc.append(perf_metric(metric1, y_test, y_test_hat))
    mlp_arousal_auc.append(perf_metric(metric2, y_test, y_test_hat))

In supervised_models.py!
6896/6896 [==============================] - 4s 553us/step
In supervised_models.py!
6896/6896 [==============================] - 4s 555us/step
In supervised_models.py!
6896/6896 [==============================] - 4s 560us/step
In supervised_models.py!
6896/6896 [==============================] - 4s 568us/step
In supervised_models.py!
6896/6896 [==============================] - 4s 569us/step


In [19]:
np.round(mlp_arousal_acc,4)

array([0.8556, 0.8592, 0.8568, 0.8567, 0.8585])

In [20]:
np.round(mlp_arousal_auc, 4)

array([0.924 , 0.9258, 0.9217, 0.9238, 0.9238])

In [21]:
print("Mean of arousal accuracies for MLP is ",round(np.mean(mlp_arousal_acc),4))

print("Mean of arousal accuracies AUC Score for MLP is ",round(np.mean(mlp_arousal_auc), 4))

# Compute standard deviation
std_dev_acc = np.std(mlp_arousal_acc)
print("Standard Deviation of arousal Accuracies using MLP baseline: ", std_dev_acc)

Mean of arousal accuracies for MLP is  0.8573
Mean of arousal accuracies AUC Score for MLP is  0.9238
Standard Deviation of arousal Accuracies using MLP baseline:  0.0013176702356679972
